In [71]:
!pip install tensorflow 

In [72]:
!pip install tensorflow-gpu

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "c:\Users\Stan\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\Stan\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\Stan\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\S

In [73]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [74]:
df = pd.read_csv("train.csv.zip")
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [75]:
df = df.head(5000)

In [76]:
from tensorflow.keras.layers import TextVectorization

In [77]:
X = df['comment_text']
y = df[df.columns[4:6]].values

In [78]:
print(
X.shape,
y.shape)

(5000,) (5000, 6)


In [79]:
MAX_FEATURES = 100000

In [80]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [81]:
# TextVectorization??

In [82]:
vectorizer.adapt(X.values)

In [83]:
# vectorizer('Hello is this ok?')

In [84]:
# vectorizer.get_vocabulary()

In [85]:
print(X)

0       Explanation\nWhy the edits made under my usern...
1       D'aww! He matches this background colour I'm s...
2       Hey man, I'm really not trying to edit war. It...
3       "\nMore\nI can't make any real suggestions on ...
4       You, sir, are my hero. Any chance you remember...
                              ...                        
4995    "\n\nHello Marcruhwedell, and Welcome to Wikip...
4996    ...that's why I did ....cheers,   (talk · cont...
4997    No, it's not a delayed reaction\n\nI just happ...
4998    "\n\nA slight difference with you\nI have to d...
4999    "\n\nNonsense.  The truth is NO schools give s...
Name: comment_text, Length: 5000, dtype: object


In [86]:
vectorizer_text = vectorizer(X.values)

In [87]:
vectorizer_text

<tf.Tensor: shape=(5000, 1800), dtype=int64, numpy=
array([[  602,    76,     2, ...,     0,     0,     0],
       [23487,    55,  1825, ...,     0,     0,     0],
       [  504,   412,    69, ...,     0,     0,     0],
       ...,
       [   48,    46,    15, ...,     0,     0,     0],
       [    6,  5215,   878, ...,     0,     0,     0],
       [  742,     2,   559, ...,     0,     0,     0]], dtype=int64)>

In [88]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorizer_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(5000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [89]:
# batch_X, batch_y = dataset.as_numpy_iterator().next()

In [90]:
# print(batch_X.shape, batch_y.shape)

In [91]:
# Might need to change this to match correct pertcentages when test is in a seperate file
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

Part 2

In [92]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [93]:
y.shape

(5000, 6)

In [94]:
X.shape

(5000,)

In [103]:
X_numpy = X.values  # Convert pandas Series to NumPy array

# Number of samples (rows) and features per sample (columns)
num_samples = X_numpy.shape[0]  # Number of samples
num_features_per_sample = 1  # Since X is a 1D array, each sample has 1 feature

# Reshape X to have the correct shape
X_reshaped = X_numpy.reshape(num_samples, num_features_per_sample)

# Define input shape based on the reshaped X
input_shape = X_reshaped.shape[1:]  

In [104]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer , 2 for this model, 6 for all columns
model.add(Dense(2, activation='sigmoid'))

Had to put a virtual shape inside the model, put since it's not actually X it won't predict. Can't seem to get X to should up in model.summary()

In [105]:

input_shape = (5000,)
model.build(input_shape=(None,) + input_shape)

In [106]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [107]:

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 5000, 32)       │     3,200,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 64)             │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,291,686 (12.56 MB)

 Trainable params: 3,291,686 (12.56 MB)

 Non-trainable params: 0 (0.00 B)

In [108]:
history = model.fit(train, epochs=1, validation_data=val)

TypeError: Expected float32, but got <keras.src.losses.losses.BinaryCrossentropy object at 0x00000295AC3C3990> of type 'BinaryCrossentropy'.